## **Notebook to make predictions with reprocessed Yoneda data and EFLUX2** 

By Christina Schenk and Garrett Roell

Tested on biodesign_3.7 kernel on jprime

### EFLUX2 predictions and evaluations
This notebook predicts fluxes for R. opacus cultures growing with glucose.

#### **Data Labels**:

#### Henson transcriptomics data: 
* WT 1.0 g/L Glucose (**WT-G**) (3 trials at 2 timepoints, 6 total trials)

#### Combined with Rhiannon 2018 metabolomics and OD data:
* Metabolomics and OD data for WT Glucose (**WT-G**)
                                               

### Method: 
<ol>
<li>Predict fluxes with EFLUX2</li>
<li>Compare predictions with 13CMFA: Scatter plots and flux maps</li>
<li>Load file with observed growth rates (Notebook E)</li>
<li>Save growth rate predictions to csv file</li>
</ol>



##### **Import python packages**

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cobra
import scipy.stats
#import cplex
%matplotlib inline

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from matplotlib.cbook import get_sample_data
import matplotlib.image as mpimg
import matplotlib.cm as cm

from edd_utils import login, export_study, export_metadata

from sklearn.metrics import r2_score

SourceDir = '../src'
sys.path.append(SourceDir)
from ensemblemethods import *
# from utils import *

# SourceDir = '../src'
# sys.path.append(SourceDir)
# from utils import *

output_dir = '../plots/CPM/'
from plot import *

##### **Load data**

In [2]:
# # Study to Download
# study_slug = 'biodesign_yoneda_set3_reprocessed'
# # EDD server
# edd_server = 'public-edd.jbei.org'
# user       = 'schenkch'

In [3]:
# session = login(edd_server=edd_server, user=user)

# df = export_study(session, study_slug, edd_server=edd_server)
# #df.head()

##### **Filter transcriptomics data from all EDD data into different dataframes**

In [4]:
# df_Trans = df[df['Protocol'].str.contains('Transcriptomics')]
# df_Trans.head()

In [5]:
# df_Trans = transcript_measurements['glucose_cpm'] #fpkm
# scratch/OpacusBiodesign/transcript_data/csv/henson_CPM_melted.csv
transcript_df = pd.read_csv('../transcript_data/csv/henson_CPM_melted.csv')
transcript_df.head()

,Line Name,Measurement Type,Time,Count,Units
0,WT-M-R1,WP_005263480_1,20,19.003420,CPM
1,WT-M-R2,WP_005263480_1,20,16.800668,CPM
2,WT-M-R3,WP_005263480_1,20,12.955220,CPM
3,WT-M-R1,WP_005263480_1,32,10.984975,CPM
4,WT-M-R2,WP_005263480_1,32,9.520107,CPM


In [6]:
# time could be 10 or 13
transcript_df_T1 = transcript_df[transcript_df['Time'] == 10]
transcript_df_T2 = transcript_df[transcript_df['Time'] == 13]
# transcript_df_T2 = df_trans[df_trans['Time'] == 13 ]
display(transcript_df_T1.head())
display(transcript_df_T2.head())


,Line Name,Measurement Type,Time,Count,Units
6,WT-Glu-R1,WP_005263480_1,10,27.446117,CPM
7,WT-Glu-R2,WP_005263480_1,10,27.543845,CPM
8,WT-Glu-R3,WP_005263480_1,10,26.797758,CPM
32,PVHG-Glu-R1,WP_005263480_1,10,24.688922,CPM
33,PVHG-Glu-R2,WP_005263480_1,10,26.027038,CPM


,Line Name,Measurement Type,Time,Count,Units
9,WT-Glu-R1,WP_005263480_1,13,15.584245,CPM
10,WT-Glu-R2,WP_005263480_1,13,22.444427,CPM
11,WT-Glu-R3,WP_005263480_1,13,20.433310,CPM
35,PVHG-Glu-R1,WP_005263480_1,13,22.993761,CPM
36,PVHG-Glu-R2,WP_005263480_1,13,23.797461,CPM


##### **Load Genome Scale Model**

In [7]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated.xml")

##### **Define a dictionary to hold detailed data**

In [8]:
details = {}

##### **Get FBA Fluxes for Glucose**

In [9]:
fbasol_glu =  FBA_pred(model, 'glucose') 
fbasol_glu.fluxes.head()

# Add glucose flux values to data frame
details['name'] = {}
details['reaction'] = {}
details['glu FBA flux'] = {}
for rxn_id, fba_flux in fbasol_glu.fluxes.items():
    details['name'][rxn_id] = model.reactions.get_by_id(rxn_id).name
    details['reaction'][rxn_id] = model.reactions.get_by_id(rxn_id).reaction
    details['glu FBA flux'][rxn_id] = fba_flux

details_df = pd.DataFrame(details)
details_df.head()

{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 100,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

,name,reaction,glu FBA flux
12DGR140tipp,"1,2 diacylglycerol transport via flipping (per...",12dgr140_p --> 12dgr140_c,0.0
13PPDH,"1,3-propanediol dehydrogenase",3hppnl_c + h_c + nadh_c <=> 13ppd_c + nad_c,0.0
1P2CBXLCYCL,1 Pyrroline 2 carboxylate cyclation,5a2opntn_c <=> 1p2cbxl_c + h2o_c + h_c,0.0
1P2CBXLR,Delta1 piperideine 2 carboxylate reductase,1p2cbxl_c + 2.0 h_c + nadph_c --> nadp_c + pro...,0.0
23CTI1,Decenyl coa cis trans isomerization cis dec 3...,decoa_c --> dc2coa_c + h_c,0.0


##### **Get pFBA Fluxes for Glucose**

In [10]:
pfbasol_glu =  pFBA_pred(model, 'glucose') 

details['glu pFBA flux'] = {}
for rxn_id, pfba_flux in pfbasol_glu.fluxes.items():
    details['glu pFBA flux'][rxn_id] = pfba_flux

details_df = pd.DataFrame(details)
details_df.head()

{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 100,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

,name,reaction,glu FBA flux,glu pFBA flux
12DGR140tipp,"1,2 diacylglycerol transport via flipping (per...",12dgr140_p --> 12dgr140_c,0.0,0.0
13PPDH,"1,3-propanediol dehydrogenase",3hppnl_c + h_c + nadh_c <=> 13ppd_c + nad_c,0.0,0.0
1P2CBXLCYCL,1 Pyrroline 2 carboxylate cyclation,5a2opntn_c <=> 1p2cbxl_c + h2o_c + h_c,0.0,0.0
1P2CBXLR,Delta1 piperideine 2 carboxylate reductase,1p2cbxl_c + 2.0 h_c + nadph_c --> nadp_c + pro...,0.0,0.0
23CTI1,Decenyl coa cis trans isomerization cis dec 3...,decoa_c --> dc2coa_c + h_c,0.0,0.0


##### **Get Transcript Values for all 6 Glucose Conditions**

In [13]:
for time in ['T1', 'T2']:
    for trial in ['WT-Glu-R1', 'WT-Glu-R2','WT-Glu-R3']:
        condition = f'{trial}-{time}'
        print(condition)
        
        details[f'{condition} transcripts'] = {}

        transcriptomics = []


        if time == 'T1':
            transcriptomics = transcript_df_T1[transcript_df_T1['Line Name'] == trial]
        else:
            transcriptomics = transcript_df_T2[transcript_df_T2['Line Name'] == trial]
            
            
        trans_data = construct_trans_df(transcriptomics, trial)
        
#         display(trans_data.head())

        # this is based on eflux function
        for rxn in model.reactions:
            if 'EX_' not in str(rxn.id):
                if rxn.gene_reaction_rule:
                    details[f'{condition} transcripts'][str(rxn.id)] = transcript_value_for_rxn(model, trans_data, rxn)
            else:
                """When there is no GPR, the arbitrary bounds are removed. 
                Common arbitrary bound value of 1000 for E.coli, might be different depending on the model, e.g., 99999.0 for iMM904 yeast model in BiGG"""
                details[f'{condition} transcripts'][str(rxn.id)]  = 1000

WT-Glu-R1-T1


,Count
Measurement Type,
WP_005263480_1,27.446117
WP_005249107_1,355.352526
WP_005249106_1,131.395949
WP_005249105_1,64.507709
WP_005249104_1,42.802872


WT-Glu-R2-T1


,Count
Measurement Type,
WP_005263480_1,27.543845
WP_005249107_1,349.533393
WP_005249106_1,131.650242
WP_005249105_1,52.886850
WP_005249104_1,35.146746


WT-Glu-R3-T1


...

,Count
Measurement Type,
WP_005263480_1,15.584245
WP_005249107_1,219.798570
WP_005249106_1,147.746736
WP_005249105_1,57.985534
WP_005249104_1,49.181448


WT-Glu-R2-T2


,Count
Measurement Type,
WP_005263480_1,22.444427
WP_005249107_1,328.518773
WP_005249106_1,124.571695
WP_005249105_1,48.937050
WP_005249104_1,37.151164


WT-Glu-R3-T2


,Count
Measurement Type,
WP_005263480_1,20.433310
WP_005249107_1,239.739580
WP_005249106_1,80.325988
WP_005249105_1,41.992422
WP_005249104_1,26.568932


In [16]:
# details
details_df = pd.DataFrame(details)
details_df = details_df.sort_values(by=['glu pFBA flux'], ascending=False)

details_df.head(10)

,name,reaction,glu FBA flux,glu pFBA flux,WT-Glu-R1-T1 transcripts,WT-Glu-R2-T1 transcripts,WT-Glu-R3-T1 transcripts,WT-Glu-R1-T2 transcripts,WT-Glu-R2-T2 transcripts,WT-Glu-R3-T2 transcripts
EX_h2o_e,R_EX_h2o_e,h2o_e <=>,551.928910,551.911916,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
O2tex,Oxygen transport via diffusion (extracellular ...,o2_e <=> o2_p,444.283499,375.998325,inf,inf,inf,inf,inf,inf
O2tpp,O2 transport via diffusion (periplasm),o2_p <=> o2_c,444.283499,375.998325,inf,inf,inf,inf,inf,inf
EX_co2_e,R_EX_co2_e,co2_e -->,444.886966,308.333611,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
NADHPO,NADH peroxidase,h2o2_c + h_c + nadh_c --> 2.0 h2o_c + nad_c,17.711127,301.785025,51.811547,65.891813,45.794799,66.081246,92.852288,39.628238
NOX,NADH oxidase,h_c + nadh_c + o2_c --> h2o2_c + nad_c,0.000000,284.059938,10.082247,7.402825,11.379870,6.780158,9.889896,8.105776
GAPD,Glyceraldehyde-3-phosphate dehydrogenase,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c,180.219701,180.219701,2564.858265,4303.709097,1668.343994,1608.314303,4601.158810,3910.079942
ENO,Enolase,2pg_c <=> h2o_c + pep_c,172.336396,172.331984,3043.531608,2406.385019,2494.669388,3212.378357,3598.231120,2553.431990
PDH,Pyruvate dehydrogenase,coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...,152.587264,151.905268,4046.808529,3758.967873,3024.383978,2358.483178,4381.890080,3468.371445
FUM,Fumarase,fum_c + h2o_c <=> mal__L_c,140.064898,140.060487,608.949042,686.195207,549.996455,518.125542,705.923353,795.491841


##### **Get E-Flux Values for each condition**

In [17]:
for time in ['T1', 'T2']:
    for trial in ['WT-Glu-R1', 'WT-Glu-R2','WT-Glu-R3']:
        condition = f'{trial}-{time}'
        print(condition)
        
        details[f'{condition} E-Flux2 flux'] = {}

        transcriptomics = []


        if time == 'T1':
            transcriptomics = transcript_df_T1[transcript_df_T1['Line Name'] == trial]
        else:
            transcriptomics = transcript_df_T2[transcript_df_T2['Line Name'] == trial]


        trans_data = construct_trans_df(transcriptomics, trial)
        
        eflux2_sol = eflux2_pred(model, trans_data, trial, 'glucose', sub_uptake_rate=100)
        
        for rxn_id, eflux2_flux in eflux2_sol.fluxes.items():
            details[f'{condition} E-Flux2 flux'][rxn_id] = eflux2_flux
            
        details_df = pd.DataFrame(details)
        display(details_df.head())

WT-Glu-R1-T1
FBA status optimal
FBA solution 4.486259051815464


{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

EFlux2 status optimal
EFlux2 solution 2118327.126076317


,name,reaction,glu FBA flux,glu pFBA flux,WT-Glu-R1-T1 transcripts,WT-Glu-R2-T1 transcripts,WT-Glu-R3-T1 transcripts,WT-Glu-R1-T2 transcripts,WT-Glu-R2-T2 transcripts,WT-Glu-R3-T2 transcripts,WT-Glu-R1-T1 E-Flux2 flux
12DGR140tipp,"1,2 diacylglycerol transport via flipping (per...",12dgr140_p --> 12dgr140_c,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13PPDH,"1,3-propanediol dehydrogenase",3hppnl_c + h_c + nadh_c <=> 13ppd_c + nad_c,0.0,0.0,38.415228,43.349877,33.313651,34.507971,43.300322,37.658084,0.0
1P2CBXLCYCL,1 Pyrroline 2 carboxylate cyclation,5a2opntn_c <=> 1p2cbxl_c + h2o_c + h_c,0.0,0.0,inf,inf,inf,inf,inf,inf,0.0
1P2CBXLR,Delta1 piperideine 2 carboxylate reductase,1p2cbxl_c + 2.0 h_c + nadph_c --> nadp_c + pro...,0.0,0.0,0.840187,0.333461,1.193051,1.416750,1.383561,0.788062,0.0
23CTI1,Decenyl coa cis trans isomerization cis dec 3...,decoa_c --> dc2coa_c + h_c,0.0,0.0,23.058472,26.343387,24.411656,21.352439,22.136969,33.267455,0.0


WT-Glu-R2-T1
FBA status optimal
FBA solution 5.659970353495349


...

,name,reaction,glu FBA flux,glu pFBA flux,WT-Glu-R1-T1 transcripts,WT-Glu-R2-T1 transcripts,WT-Glu-R3-T1 transcripts,WT-Glu-R1-T2 transcripts,WT-Glu-R2-T2 transcripts,WT-Glu-R3-T2 transcripts,WT-Glu-R1-T1 E-Flux2 flux,WT-Glu-R2-T1 E-Flux2 flux,WT-Glu-R3-T1 E-Flux2 flux,WT-Glu-R1-T2 E-Flux2 flux,WT-Glu-R2-T2 E-Flux2 flux
12DGR140tipp,"1,2 diacylglycerol transport via flipping (per...",12dgr140_p --> 12dgr140_c,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.00000,0.0
13PPDH,"1,3-propanediol dehydrogenase",3hppnl_c + h_c + nadh_c <=> 13ppd_c + nad_c,0.0,0.0,38.415228,43.349877,33.313651,34.507971,43.300322,37.658084,0.0,0.0,0.0,0.00000,0.0
1P2CBXLCYCL,1 Pyrroline 2 carboxylate cyclation,5a2opntn_c <=> 1p2cbxl_c + h2o_c + h_c,0.0,0.0,inf,inf,inf,inf,inf,inf,0.0,0.0,0.0,0.00000,0.0
1P2CBXLR,Delta1 piperideine 2 carboxylate reductase,1p2cbxl_c + 2.0 h_c + nadph_c --> nadp_c + pro...,0.0,0.0,0.840187,0.333461,1.193051,1.416750,1.383561,0.788062,0.0,0.0,0.0,1.41675,0.0
23CTI1,Decenyl coa cis trans isomerization cis dec 3...,decoa_c --> dc2coa_c + h_c,0.0,0.0,23.058472,26.343387,24.411656,21.352439,22.136969,33.267455,0.0,0.0,0.0,0.00000,0.0


WT-Glu-R3-T2
FBA status optimal
FBA solution 7.160901262016445


{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

EFlux2 status optimal
EFlux2 solution 5838880.895895321


,name,reaction,glu FBA flux,glu pFBA flux,WT-Glu-R1-T1 transcripts,WT-Glu-R2-T1 transcripts,WT-Glu-R3-T1 transcripts,WT-Glu-R1-T2 transcripts,WT-Glu-R2-T2 transcripts,WT-Glu-R3-T2 transcripts,WT-Glu-R1-T1 E-Flux2 flux,WT-Glu-R2-T1 E-Flux2 flux,WT-Glu-R3-T1 E-Flux2 flux,WT-Glu-R1-T2 E-Flux2 flux,WT-Glu-R2-T2 E-Flux2 flux,WT-Glu-R3-T2 E-Flux2 flux
12DGR140tipp,"1,2 diacylglycerol transport via flipping (per...",12dgr140_p --> 12dgr140_c,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.00000,0.0,0.0
13PPDH,"1,3-propanediol dehydrogenase",3hppnl_c + h_c + nadh_c <=> 13ppd_c + nad_c,0.0,0.0,38.415228,43.349877,33.313651,34.507971,43.300322,37.658084,0.0,0.0,0.0,0.00000,0.0,0.0
1P2CBXLCYCL,1 Pyrroline 2 carboxylate cyclation,5a2opntn_c <=> 1p2cbxl_c + h2o_c + h_c,0.0,0.0,inf,inf,inf,inf,inf,inf,0.0,0.0,0.0,0.00000,0.0,0.0
1P2CBXLR,Delta1 piperideine 2 carboxylate reductase,1p2cbxl_c + 2.0 h_c + nadph_c --> nadp_c + pro...,0.0,0.0,0.840187,0.333461,1.193051,1.416750,1.383561,0.788062,0.0,0.0,0.0,1.41675,0.0,0.0
23CTI1,Decenyl coa cis trans isomerization cis dec 3...,decoa_c --> dc2coa_c + h_c,0.0,0.0,23.058472,26.343387,24.411656,21.352439,22.136969,33.267455,0.0,0.0,0.0,0.00000,0.0,0.0


##### **Sort reactions by FBA flux values**

In [18]:
details_df = details_df.sort_values(by=['glu pFBA flux'], ascending=False)
details_df.head()

,name,reaction,glu FBA flux,glu pFBA flux,WT-Glu-R1-T1 transcripts,WT-Glu-R2-T1 transcripts,WT-Glu-R3-T1 transcripts,WT-Glu-R1-T2 transcripts,WT-Glu-R2-T2 transcripts,WT-Glu-R3-T2 transcripts,WT-Glu-R1-T1 E-Flux2 flux,WT-Glu-R2-T1 E-Flux2 flux,WT-Glu-R3-T1 E-Flux2 flux,WT-Glu-R1-T2 E-Flux2 flux,WT-Glu-R2-T2 E-Flux2 flux,WT-Glu-R3-T2 E-Flux2 flux
EX_h2o_e,R_EX_h2o_e,h2o_e <=>,551.928910,551.911916,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,602.724147,909.666733,610.570915,942.400379,790.960901,1000.000000
O2tex,Oxygen transport via diffusion (extracellular ...,o2_e <=> o2_p,444.283499,375.998325,inf,inf,inf,inf,inf,inf,471.022482,766.950870,475.031815,653.207604,600.993425,789.329283
O2tpp,O2 transport via diffusion (periplasm),o2_p <=> o2_c,444.283499,375.998325,inf,inf,inf,inf,inf,inf,471.022482,766.950870,475.031815,653.207604,600.993425,789.329283
EX_co2_e,R_EX_co2_e,co2_e -->,444.886966,308.333611,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,471.721730,766.947099,478.538886,748.751564,605.179666,793.587827
NADHPO,NADH peroxidase,h2o2_c + h_c + nadh_c --> 2.0 h2o_c + nad_c,17.711127,301.785025,51.811547,65.891813,45.794799,66.081246,92.852288,39.628238,51.811547,65.891813,45.794799,66.081246,92.852288,39.628238


##### **Save data as a csv**

In [19]:
details_df.to_csv('../detailed_data/EFlux_glucose_details.csv')

In [20]:
model.reactions.get_by_id('UGT_BS')

Reaction identifier,UGT_BS
Name,UDP glucosyltransferase diglucosyl
Memory address,0x07f9dc9978310
Stoichiometry,0.01 12dgr_BS_c + 2.0 udpg_c --> 0.01 d12dg_BS_c + 2.0 h_c + 2.0 udp_c 0.01 1 2 diacylglycerol C3436H6672O500 + 2.0 UDPglucose --> 0.01 Diglucosyl 1 2 diacylglycerol C4636H8672O1500 + 2.0 H+ + 2.0 UDP C9H11N2O12P2
GPR,
Lower bound,0.0
Upper bound,1000.0


# Delete the rest of the notebook under this

#### **1. EFLUX2 Predictions for Wild type**

In [16]:
# eflux2sol, eflux2sol_std = eflux2_pred_for_three_reps(model, df_trans, 'WT-Glu-R1', 'WT-Glu-R2','WT-Glu-R3', 'glucose')

##### **Save solution to data frame**

In [17]:
# eflux2soldf = pd.DataFrame(eflux2sol, columns=['fluxes'])
# eflux2solstddf = pd.DataFrame(eflux2sol_std, columns=['stds'])

#### **2. Plot solutions: Comparison of EFLUX2 WT predictions and 13c measurements**

##### **Load 13cdata**

#####  **Get 13C MFA measured fluxes for glucose**

In [18]:
# glucose_fluxes = pd.read_csv('../13C_flux_data/13C_glucose_flux_data.csv')

# # Remove rows that do not have a mapping to the GSM
# glucose_fluxes.dropna(subset = ["Forward Reactions"], inplace=True)
# print(f'There are {len(glucose_fluxes)} fluxes that can be compared between the MFA and FBA')
# glucose_fluxes

##### **Add Glucose EFLUX2 flux values to phenol fluxes dataframe**

In [19]:
# obspred_fluxes = add_pred_fluxes_to_13c_df(glucose_fluxes, eflux2soldf, eflux2solstddf, 'glucose', 'E-Flux2', 'WT')
# obspred_fluxes.to_csv('../13C_flux_data/obspredfluxes_Glucose_EFLUX2_CPM.csv', index=True, header= True)
# obspred_fluxes.head()

### **Plot EFLUX2 vs 13C MFA**

##### **Plot 13C MFA observations vs. EFLUX2 predictions**

##### Scatter plot: 13CMFA vs. EFLUX2

In [20]:
# scatterplotcomp_obs_vs_pred(obspred_fluxes, substrate='glucose', method='E-Flux2', strain='WT')

##### **Plot 13C MFA observations vs. EFLUX2 predictions with confidence intervals**

##### Scatter plot with standard deviations: 13CMFA vs. EFLUX2

In [21]:
# scatterplotcomp_obs_vs_pred_withstd(obspred_fluxes, substrate='glucose', method='E-Flux2', strain='WT')

#### **Glucose EFlux2 WT Flux Map**

In [22]:
# map_flux_results(obspred_fluxes, 'E-Flux2 WT Value')

### **3. Load File with observed growth rates**

##### **Load observed growth rates and plot glucose growth rates**

In [23]:
# consumption_and_growth_data = pd.read_csv('../consumption_and_growth_data/consumption_and_growth_data.csv', index_col=0)
# consumption_and_growth_data

##### **For comparison of predicted and observed growth rates: scale predicted growth rate by multiplying with (observed substrate uptake / predicted substrate uptake)**

In [24]:
# scaledgrowthrate_wtlng = scale_growth_to_sub(
#     eflux2soldf.loc['Growth_Glucose',:].values[0], 
#     eflux2soldf.loc['EX_glc__D_e',:].values[0],
#     consumption_and_growth_data.loc['WT-G', 'substrate consumption rate']
# )

##### **Add scaled values to new dataframe**

In [25]:
# allgrowthrates=pd.DataFrame(index=['WT-G'], columns=['Growth_Glucose_EFLUX2'], dtype=float)
# allgrowthrates.at['WT-G','Growth_Glucose_EFLUX2'] = scaledgrowthrate_wtlng
# allgrowthrates

### **4. Save growth rate predictions to csv file**

##### **Save growth rates as csv file**

In [26]:
# allgrowthrates.to_csv('../consumption_and_growth_data/allgrowthratesGlucoseEFLUX2_CPM.csv', index=True, header= True)